In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

In [2]:
os.chdir("..")
print(os.getcwd())

c:\Users\valen\Desktop\etl_workshop002


In [3]:
lastfm_df = pd.read_csv("data/raw/lastfm_data.csv")
lastfm_df.head()

,artist_name,listeners,playcount,similar
0,!nvite,8030,19556,Bayleaf Manilla;Smiloh;Tentoe
1,#Kids,815,4431,Lullabies In Nature;Modern Children's Songs;Zo...
2,$affie,12512,32051,Ricki Wu;Lobby Lingu!n!;Exxecs
3,&ME,138173,1164546,Rampa;Adriatique;Adam Port
4,'Falsettos' 2016 Broadway Company,21687,487458,Anthony Rosenthal;Stephanie J. Block;Christian...


In [4]:
artists_df=pd.read_csv("data/raw/artists.csv")
needed_artists = artists_df['artist'].tolist()  

In [5]:
found_artists = lastfm_df["artist_name"].unique()
missing = set(needed_artists) - set(found_artists)

print("Artists that do not yet appear after mapping:")
print(missing)

Artists that do not yet appear after mapping:
{'Maxence Cyrin', 'Clay Walker', 'Dombresky', 'Phyno', 'The Handsome Family', 'Maximilian Sievert', 'Maikel Delacalle', 'Clarence White', 'Reynaldo Santiago Chino ', 'The Kentucky Colonels', 'Maxel', 'Maunalua', 'Maxi Rozh', 'Chrystian & Ralf', 'Maxida Märak'}


In [6]:
len(missing)

15

In [7]:
lastfm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17632 entries, 0 to 17631
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  17632 non-null  object
 1   listeners    17632 non-null  int64 
 2   playcount    17632 non-null  int64 
 3   similar      17199 non-null  object
dtypes: int64(2), object(2)
memory usage: 551.1+ KB


In [8]:
# Análisis de patrones en artistas sin similares
similar_nulls = lastfm_df[lastfm_df['similar'].isnull()]
print("\nArtistas sin similares pero con alta popularidad:")
similar_nulls[similar_nulls['playcount'] > 5000][['artist_name', 'playcount', 'listeners']]


Artistas sin similares pero con alta popularidad:


,artist_name,playcount,listeners
16,102 Boyz,1628742,40763
46,220 KID,934292,134535
50,24kGoldn,20706450,1031742
73,3TEETH,3099647,145351
94,6 Dogs,7886152,263628
...,...,...,...
17318,slipmami,8652317,163595
17328,suggi,2474663,150887
17371,yeule,23622541,495430
17429,Автоспорт,712217,49131


In [9]:
lastfm_df['similar'].str.count(';').value_counts()

similar
2.0    17191
0.0        3
3.0        3
1.0        1
5.0        1
Name: count, dtype: int64

Renaming columns

In [10]:
lastfm_df["artist"]=lastfm_df["artist_name"]
lastfm_df["lastfm_listeners"]=lastfm_df["listeners"]
lastfm_df["lastfm_playcount"]=lastfm_df["playcount"]

In [11]:
lastfm_df['similar'] = lastfm_df['similar'].replace('', np.nan)

split_similar = lastfm_df['similar'].dropna().str.split(';', n=2, expand=True)

split_similar = split_similar.apply(lambda col: col.str.strip())

lastfm_df[['similar_1', 'similar_2', 'similar_3']] = split_similar

In [ ]:
lastfm_df['artist'] = (
    lastfm_df['artist']
    .str.strip()  
    .str.replace(r'[\"]', '', regex=True) 
    .str.replace(r'\s+', ' ', regex=True) 
)

In [13]:
lastfm_df.drop(columns=["artist_name", "similar", "listeners", "playcount"], inplace= True)

In [14]:
lastfm_df.head()

,artist,lastfm_listeners,lastfm_playcount,similar_1,similar_2,similar_3
0,!nvite,8030,19556,Bayleaf Manilla,Smiloh,Tentoe
1,#Kids,815,4431,Lullabies In Nature,Modern Children's Songs,Zouzounia TV
2,$affie,12512,32051,Ricki Wu,Lobby Lingu!n!,Exxecs
3,&ME,138173,1164546,Rampa,Adriatique,Adam Port
4,'Falsettos' 2016 Broadway Company,21687,487458,Anthony Rosenthal,Stephanie J. Block,Christian Borle


In [15]:
lastfm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17632 entries, 0 to 17631
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artist            17632 non-null  object
 1   lastfm_listeners  17632 non-null  int64 
 2   lastfm_playcount  17632 non-null  int64 
 3   similar_1         17199 non-null  object
 4   similar_2         17196 non-null  object
 5   similar_3         17195 non-null  object
dtypes: int64(2), object(4)
memory usage: 826.6+ KB


In [16]:
lastfm_df.to_csv("data/clean/lastfm_artists.csv", index=False)